# MDTraj atom selection

## MDTrajでの原子選択

以下の文書は主に MDTrajのドキュメント https://www.mdtraj.org/1.9.5/examples/atom-selection.html を日本語訳したものです。

MDTrajでの原子と残基の選択の基本について説明します。まず、例のトラジェクトリをロードします。

In [ ]:
import mdtraj as md
import numpy as np

# DCDファイルからトラジェクトリを読み込み、対応するPDBトポロジーファイルを使用する
# トラジェクトリオブジェクト traj が作成される
traj = md.load('../md/3_production.dcd', top='../md/system.pdb')

# traj の中身を出力する
print(traj)

traj.n_atoms や traj.n_residues を使用して、より直接的に原子数や残基数を確認することもできます。

In [ ]:
print('原子は何個？ %s' % traj.n_atoms)
print('残基は何個？ %s' % traj.n_residues)

また、traj.xyz を操作することで原子の位置を操作することもできます。これは、各原子のxyz座標を含むNumPy配列で、次元は(n_frames, n_atoms, 3)です。5フレーム目の10番目の原子の3D座標を探してみましょう。

In [ ]:
frame_idx = 4 # ゼロから始まるフレーム番号
atom_idx = 9 # ゼロから始まる原子インデックス
print('10フレーム目の5番目の原子はどこにある？')
print('x: %s\ty: %s\tz: %s' % tuple(traj.xyz[frame_idx, atom_idx,:]))

トポロジーオブジェクト、すべてのTrajectoryオブジェクトにはTopologyが含まれています。TrajectoryのTopologyには、系（分子）の結合情報や特定のChain, Residue(残基), Atom情報が含まれています。

In [ ]:
topology = traj.topology
print(topology)

トポロジーオブジェクトを使用すれば、特定の原子を選択したり、すべての原子をループしたりすることができます。 (注意: zero-based indexです)

In [ ]:
print('5番目の原子: %s' % topology.atom(4))
print('すべての原子: %s' % [atom for atom in topology.atoms])

Residue(残基)についても同じことが言えます。

In [ ]:
print('2番目の残基: %s' % traj.topology.residue(1))
print('すべての残基: %s' % [residue for residue in traj.topology.residues])

さらに、すべての原子と残基もオブジェクトであり、それぞれが独自のpropertyを持っています。以下は、そのうちのいくつかを説明するシンプルな例です。

In [ ]:
atom = topology.atom(10)
print('''こんにちは！私は %s 番目の原子で、名前は %s です。
私は %s 原子で、%s 個の結合を持っています。
私は %s 残基の一部です。''' % (atom.index, atom.name, atom.element.name, atom.n_bonds, atom.residue.name))

また、atom.is_sidechainやresidue.is_proteinのようなより複雑なプロパティもあります。これらを利用することで、よりパワフルな選択が可能になります。

これらのプロパティをPythonのフィルタリスト機能と組み合わせると強力です。たとえば、分子の側鎖にあるすべての Carbon atom のインデックスを求めたいとしましょう。こんな感じで実現できます。

In [ ]:
print([atom.index for atom in topology.atoms if atom.element.symbol == 'C' and atom.is_sidechain])

または、最初の Chain のすべての 偶数インデックスの Residue (残基) を求めることもできます（ただし、この例では1つの鎖しかありません）

In [ ]:
print([residue for residue in topology.chain(0).residues if residue.index % 2 == 0])

上記のようなフィルタリストのプログラミングに迷っている場合、MDTrajはPyMolやVMDに似た豊富な原子選択言語も提供しています。これには、topology.selectを使用してアクセスできます。最後の2つの残基のすべての原子を探してみましょう。

原子選択構文の詳細については、[ドキュメント](https://www.mdtraj.org/1.9.7/atom_selection.html)をご覧ください。

In [ ]:
print(topology.select('resid 1 to 2'))

より複雑な選択も可能です。以下では、主鎖のすべての Nitrogen atom を選択します。

In [ ]:
print(topology.select('name N and backbone'))

これらの結果を生成するコードを見たい場合は、select_expressionを使用できます。これは、atom selection コードの文字列表現を返します。

In [ ]:
selection = topology.select_expression('name CA and resid 1 to 2')
print(selection)